In [27]:
import numpy as np
from sympy import Eq, symbols, latex, I
from IPython.display import display, Math

def metodo_gauss_seidel(w, sigma, u, e, Eo, alpha_inicial=1.0, beta_inicial=1.0, max_iter=1000, tol=1e-6):
    tan = sigma / (w * e)
    alpha = alpha_inicial
    beta = beta_inicial
    
    # Verificar si es un buen conductor
    if tan > 25:  # sigma / (w * e) >> 1
        alpha = beta = np.sqrt((u * w * sigma)/(2))
        eta = np.pi/4
    else:
        for i in range(max_iter):
            # Guardar valores antiguos para verificar la convergencia
            alpha_antiguo = alpha
            beta_antiguo = beta

            # Actualizar alpha usando el valor actual de beta
            alpha = w * sigma * u / (2 * beta)
            
            # Actualizar beta usando el nuevo valor de alpha
            beta = (alpha**2 + w**2 * u * e)**0.5

            # Verificar la convergencia
            if abs(alpha - alpha_antiguo) < tol and abs(beta - beta_antiguo) < tol:
                eta = 0.5*np.arctan(tan)
                break

    
    # Variables fuera de la condición
    velocidad_propagacion = w/beta
    longitud_onda = (2*np.pi*velocidad_propagacion)/w
    modulo_impedancia = (np.sqrt(u/e))/((1+tan**2)**0.25)
    impedancia_real = modulo_impedancia*np.cos(eta)
    impedancia_imaginaria = modulo_impedancia*np.sin(eta)
    ecuacion_E = "E(z, t) = {} \cdot e^{{-{} z}} \cdot \cos({}t-{} z) \mathbf{{ax}}".format(
        Eo, round(alpha, 4), round(w, 4), round(beta, 4))
    ecuacion_H = "H(z, t) = {} \cdot e^{{-{} z}} \cdot \cos({}t-{} z - {}) \mathbf{{ay}}".format(
        round(Eo/modulo_impedancia, 4), round(alpha, 4), round(w, 4), round(beta, 4), round(eta, 4))

    
    return {
        'alpha': alpha,
        'beta': beta,
        'velocidad_propagacion': velocidad_propagacion,
        'longitud_onda': longitud_onda,
        'impedancia_real': impedancia_real,
        'impedancia_imaginaria': impedancia_imaginaria,
        'ecuacion_E': ecuacion_E,
        'ecuacion_H': ecuacion_H
    }

def imprimir_resultados(resultados):
    display(Math(r'\alpha = {} \, \text{{Np/m}}'.format(resultados['alpha'])))
    display(Math(r'\beta = {} \, \text{{rad/m}}'.format(resultados['beta'])))
    display(Math(r'v_p = {} \, \text{{m/s}}'.format(resultados['velocidad_propagacion'])))
    display(Math(r'\lambda = {} \, \text{{m}}'.format(resultados['longitud_onda'])))
    display(Math(r'\eta = {} + j{} \, \Omega'.format(resultados['impedancia_real'], resultados['impedancia_imaginaria'])))
    display(Math(resultados['ecuacion_E'] + r' \, \text{{V/m}}'))
    display(Math(resultados['ecuacion_H'] + r' \, \text{{A/m}}'))

# Prueba Dielectrico con perdidas
Eo = 5
w = 100*2*np.pi*1e6
sigma = 0.025
u = 4*np.pi*1e-7
e = (91.3*1e-9)/(36*np.pi)

print("Resultados para dieléctrico con pérdidas")
resultados_dielectrico = metodo_gauss_seidel(w, sigma, u, e, Eo)
imprimir_resultados(resultados_dielectrico)
print("\n\n")


# Prueba buen conductor
Eo = 4
w = 1e8
sigma = 3
u = 20*4*np.pi*1e-7
e = (1e-9)/(36*np.pi)

print("Resultados para un buen conductor")
resultados_conductor = metodo_gauss_seidel(w, sigma, u, e, Eo)
imprimir_resultados(resultados_conductor)

Resultados para dieléctrico con pérdidas


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>